In [7]:
# -*- coding: utf-8 -*-
"""
bmc2_BERT.ipynb

Automatically generated code for fine-tuning a BERT encoder–decoder model on the
b-mc2/sql-create-context dataset.
"""

# Install necessary libraries
!pip install transformers datasets evaluate rouge_score

# Import required modules
from datasets import load_dataset
from transformers import (
    EncoderDecoderModel,
    BertTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

from transformers.trainer_pt_utils import LabelSmoother
import evaluate
import numpy as np
import os
from transformers.trainer_utils import get_last_checkpoint

# -----------------------------------------------------------
# Step 1: Data Preparation
# -----------------------------------------------------------
# Load the SQL generation dataset from Hugging Face.
dataset = load_dataset("b-mc2/sql-create-context")

# Split the training data into training and validation sets (80/20 split)
split_dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

# -----------------------------------------------------------
# Step 2: Model and Tokenizer Initialization
# -----------------------------------------------------------
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Create an encoder–decoder model using bert-base-uncased for both encoder and decoder.
model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

# IMPORTANT: Update the decoder configuration so that it acts as a decoder.
model.decoder.config.is_decoder = True
model.decoder.config.add_cross_attention = True

# Set special tokens for the decoder.
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

# Optionally, set generation parameters.
model.config.max_length = 128
model.config.min_length = 10
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True

# -----------------------------------------------------------
# Step 3: Preprocessing & Tokenization
# -----------------------------------------------------------
# Preprocessing function to:
# 1. Combine 'context' and 'question' to form the input.
# 2. Use 'answer' as the target (SQL query).
def preprocess_function(examples):
    # Combine the context and question into one input string.
    inputs = [c + " " + q for c, q in zip(examples["context"], examples["question"])]
    targets = examples["answer"]

    # Tokenize inputs with maximum length, truncation, and padding.
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize target texts (SQL queries).
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    # Add the tokenized targets under the key "labels".
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to both training and validation datasets.
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# -----------------------------------------------------------
# Step 4: Data Collator
# -----------------------------------------------------------
# The DataCollatorForSeq2Seq pads the inputs and labels to the maximum length in the batch.
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# -----------------------------------------------------------
# Step 5: Fine-Tuning Setup
# -----------------------------------------------------------
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_bert",
    evaluation_strategy="epoch",
    save_strategy="epoch",        # checkpointing each epoch
    save_total_limit=2,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=True,
    generation_num_beams = 4,
)

# -----------------------------------------------------------
# Step 6: Evaluation Metrics Setup (BLEU and ROUGE)
# -----------------------------------------------------------
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Replace -100 (ignore index) with the tokenizer's pad token id.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode the predictions and labels into text.
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Load evaluation metrics.
    bleu_metric = evaluate.load("bleu")
    rouge_metric = evaluate.load("rouge")

    # Compute BLEU (with references formatted as a list of lists).
    bleu = bleu_metric.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])

    # Compute ROUGE scores.
    rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

    return {
        "bleu": bleu["bleu"],
        "rouge1": rouge["rouge1"],
        "rouge2": rouge["rouge2"],
        "rougeL": rouge["rougeL"],
    }

# -----------------------------------------------------------
# Step 7: Trainer Initialization and Model Fine-Tuning
# -----------------------------------------------------------
class CustomSeq2SeqTrainer(Seq2SeqTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        How the loss is computed by Seq2SeqTrainer.
        """
        labels = inputs.get("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Initialize label_smoother if not already initialized
        if self.label_smoother is None:
            self.label_smoother = LabelSmoother(epsilon=0.1)
        # Compute loss
        loss = self.label_smoother(outputs, labels)
        return (loss, outputs) if return_outputs else loss

trainer = CustomSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Check for existing checkpoints.
last_checkpoint = None
if os.path.isdir(training_args.output_dir):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)

if last_checkpoint is not None:
    print(f"Resuming from checkpoint: {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("No checkpoint found. Starting training from scratch.")
    trainer.train()



Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.e

No checkpoint found. Starting training from scratch.


/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel
1,1.428300,1.494193,0.845067,0.943652,0.894848,0.935149


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1581: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be 

ValueError: The generation config instance is invalid -- `.validate()` throws warnings and/or exceptions. Fix these issues to save the configuration.

Thrown during validation:
[UserWarning('`num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.')]

In [ ]:
# Save the final trained model.
trainer.save_model("./results_bert")

# -----------------------------------------------------------
# Step 8: Model Evaluation
# -----------------------------------------------------------
results = trainer.evaluate()
print("Evaluation Results:")
print(results)

# -----------------------------------------------------------
# Step 9: Inference and Generation
# -----------------------------------------------------------
# Select a few samples from the validation set to inspect the generated SQL queries.
sample_dataset = val_dataset.select(range(5))
predictions = trainer.predict(sample_dataset)

# Decode predictions and corresponding ground truth labels.
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)

# Display results for comparison.
for i, (pred, label) in enumerate(zip(decoded_preds, decoded_labels)):
    print(f"\nSample {i + 1}:")
    print("Generated SQL Query:", pred)
    print("Ground Truth SQL Query:", label)